# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Загрузка датасетов

In [2]:
df = pd.read_csv(
    "market_logs.log",
    sep=r"mLog: market trd at | with side |, price |, traded volume | then book became |@|x",
    names=['', 'time', 'side', 'price', 'volume', 'volume_bid', 'bid_price', 'ask_price', 'volume_ask'],
    usecols=['time', 'side', 'price', 'volume', 'volume_bid', 'bid_price', 'ask_price', 'volume_ask'],
    engine='python'
)

df2 = pd.read_csv(
    "exec_logs.log",
    sep=r"eLog: order exec at | with id |, side |, price |, traded volume |, volume left on level |, delta_execsend ",
    names=['', 'time', 'id', 'side', 'price', 'volume', 'volume_left', 'delta_execsend'],
    usecols=['time', 'id', 'side', 'price', 'volume', 'volume_left', 'delta_execsend'],
    engine='python'
)

print("df and df2")
display(df.head(), df2.head())

df and df2


,time,side,price,volume,volume_bid,bid_price,ask_price,volume_ask
0,1622505601191796236,-1,1108.9,36,6,1108.7,1108.8,104
1,1622505601222248272,-1,1108.7,17,10,1108.5,1108.6,23
2,1622505601364445668,-1,1108.5,20,16,1108.3,1108.6,23
3,1622505601405325968,1,1108.6,1,16,1108.3,1108.6,22
4,1622505601406994624,-1,1108.3,16,18,1108.2,1108.3,12


,time,id,side,price,volume,volume_left,delta_execsend
0,1622505601222248272,281481419161600,1,1108.6,1,10,30451968
1,1622505601424113404,562956395872256,-1,1108.4,1,22,17118720
2,1622505601604667896,844431372582912,1,1108.3,1,2,153866496
3,1622505602982200992,1125906349293568,-1,1108.4,1,1,153371136
4,1622505603600579508,1407381326004224,1,1108.3,1,6,618378496


### datasets info

In [3]:
df.info()
print()
print()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397802 entries, 0 to 397801
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        397802 non-null  int64  
 1   side        397802 non-null  int64  
 2   price       397802 non-null  float64
 3   volume      397802 non-null  int64  
 4   volume_bid  397802 non-null  int64  
 5   bid_price   397802 non-null  float64
 6   ask_price   397802 non-null  float64
 7   volume_ask  397802 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 24.3 MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8682 entries, 0 to 8681
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   time            8682 non-null   int64  
 1   id              8682 non-null   int64  
 2   side            8682 non-null   int64  
 3   price           8682 non-null   float64
 4   volume          8682 non-null   int6

### Проверка упорядоченности отностительно столбца `time`

In [4]:
(np.all(df.time.values[:-1] < df.time.values[1:]),
 np.all(df2.time.values[:-1] < df2.time.values[1:]))

(True, True)

# Корреляция

### Нахождение пятисекундных временных смещений `(mid_price через 5 секунд)`

In [5]:
MIN_TIME_SHIFT = int(4.5 * 10**9)


i = 0
j = 0
# Разница во времени между ордером и оценкой mid_price
# Мы стараемся посчить ровно через 5 секунд,
# но это не всегда получается из дискретности данных и перерывов.
diffs = np.empty(df2.shape[0], dtype=np.int64)
# Индексы строк из df через 5 секунд после ордера из df2
nearest = np.empty(df2.shape[0], dtype=np.int64)
while i < df2.shape[0]:
    diff = df.time[j] - df2.time[i] - df2.delta_execsend[i]
    if diff >= MIN_TIME_SHIFT:
        diffs[i] = diff
        nearest[i] = j
        i += 1
    else:
        j += 1

mid_price = df.iloc[nearest][['bid_price', 'ask_price']].mean(axis=1).values
dmid5 = (mid_price - df2.price.values) * df2.side.values



### Подсчет корреляции пирсона

In [6]:
TOP_QUANTILE = 0.217


mid_price = df.iloc[nearest][['bid_price', 'ask_price']].mean(axis=1).values
dmid5 = (mid_price - df2.price.values) * df2.side.values

print('Корреляция с выбросами:', np.corrcoef(dmid5, df2.delta_execsend)[0, 1]),
print('Корреляция без выбросов:', np.corrcoef(dmid5[diffs <= np.quantile(diffs, TOP_QUANTILE)], df2.delta_execsend.values[diffs <= np.quantile(diffs, TOP_QUANTILE)])[0, 1])
print('Верхний квантиль временного смещения измерения dmid5 (в секундах):', np.quantile(diffs, TOP_QUANTILE) / 10**9)

Корреляция с выбросами: 0.013686926395611893
Корреляция без выбросов: 0.04376968665102065
Верхний квантиль временного смещения измерения dmid5 (в секундах): 5.496720640596
